In [ ]:
# AI Hub raw json 데이터에서 bbox 추출

import os 
import shutil 
import json 
from tqdm.notebook import tqdm
import zipfile

In [ ]:
def bbox_xy2ratio(path, save_path):         # path : json파일 저장되어 있는 경로
    with open(f'{path}', 'r') as f:         # save_path : bbox.txt 파일 저장할 경로
        data = json.load(f)
    image_data = data.get('images')
    annotation_data = data.get('annotations')    # [object list]
    W = image_data['width']
    H = image_data['height']
    title = image_data.get('file_name').rstrip('.jpg')
    txt = open(f'{save_path}/{title}.txt', 'w')
    for ann in annotation_data: 
        bbox_data = ann.get('bbox')
        # print(bbox_data)
        x = (bbox_data[0]+bbox_data[2]/2)/W
        y = (bbox_data[1]+bbox_data[3]/2)/H
        w = bbox_data[2]/W
        h = bbox_data[3]/H
        damage = ann.get('damage')
        txt.write(f'{damage} {x} {y} {w} {h}\n')
    txt.close()
    pass 

In [ ]:
#drive dir
json_dir='/content/drive/MyDrive/ObjectDetection/data/160. 차량파손 이미지 데이터/01.데이터/1.Training/2.라벨링데이터/TL_damage.zip'
img_dir='/content/drive/MyDrive/ObjectDetection/data/160. 차량파손 이미지 데이터/01.데이터/1.Training/1.원천데이터/TS_damage.zip'

In [ ]:
save_dir = r'/content/drive/MyDrive/ObjectDetection/output/image_with_bbox_data'  # 이미지랑 bbox텍스트 파일 같이 저장할 경로 (내가 원하는 폴더 지정)

In [ ]:
# json 파일 있는 폴더에서 bbox txt 파일 만들어서 save_dir에 저장
for json_file in tqdm(os.listdir(json_dir)): 
    bbox_xy2ratio(os.path.join(json_dir, json_file), save_dir)

In [ ]:
# 이미지 저장된 폴더에서 이미지 복사해서 save_dir에 저장
for img in tqdm(os.listdir(img_dir)):   
    shutil.copy(os.path.join(img_dir, img), save_dir)   # 복사 말고 속도문제나 용량부족 등으로 옮겨야한다면 : shutil.copy -> shutil.move 로 대체 가능

In [ ]:
# roboflow 에서 필요한 .labels파일 만들어서 이미지, txt 파일이 저장된 폴더에 저장하기 
damage_list = ['Scratched', 'Breakage', 'Crushed', 'Separated'] # 차량 파손은 왼쪽의 4종류로 분류됨
with open(os.path.join(save_dir, 'car.labels'), 'w') as f: 
    for damage in damage_list: 
        f.write(f'{damage}\n')

In [ ]:
len(os.listdir(save_dir))   # 잘 이동 되었는지 확인 : 홀수개여야함(2*이미지 개수 + 1)

In [ ]:
# save_dir 폴더를 app.roboflow에 업로드해서 bbox잘 되어 있는지 체크
# 근데 1200개만 해도 roboflow에 업로드 시간 꽤 걸림(한 3분?)
# 그래서 몇십만개 전부 하기보다는 랜덤하게 뽑아서 해보는게 나을 수도?